# Сессия 4

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import sklearn
import pickle
import glob
from sklearn.preprocessing import StandardScaler
import codecs
pd.options.display.max_columns = 500

Импортировал модель для классификации в дальнейшем

In [2]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [3]:
df = pd.read_csv('input.csv')

Оставил только нужные атрибуты

In [4]:
df = df[['KMeans', 'properties.parent_region', 'properties.region','properties.address', 'properties.dead_count', 'properties.injured_count', 'properties.participants_count']]

In [5]:
df

,KMeans,properties.parent_region,properties.region,properties.address,properties.dead_count,properties.injured_count,properties.participants_count
0,Требует вмешательства,Алтайский край,Барнаул,"г Барнаул, пр-кт Калинина, 16В",0,4,5
1,Требует немедленного вмешательства,Алтайский край,Первомайский район,"Мостовой переход через р. Обь в г. Барнауле, 2 км",1,0,2
2,Требует немедленного вмешательства,Алтайский край,Шипуновский район,А-322 Барнаул - Рубцовск - граница с Республик...,2,2,5
3,Вмешательство не требуется,Алтайский край,Первомайский район,Р-256 Чуйский тракт Новосибирск - Барнаул - Го...,0,2,2
4,Вмешательство не требуется,Алтайский край,Барнаул,"г Барнаул, тракт Змеиногорский, 25",0,1,2
...,...,...,...,...,...,...,...
35405,Вмешательство не требуется,Тверская область,Тверь,"г Тверь, ул Шишкова, 97а",0,1,2
35406,Вмешательство не требуется,Ульяновская область,Барышский район,"с Румянцево, ул Чкалова, 137",0,1,3
35407,Вмешательство не требуется,Воронежская область,Воронеж,"г Воронеж, пр-кт Ленинский, 26 1",0,2,2
35408,Вмешательство не требуется,Воронежская область,Терновский район,"ТЕРНОВКА - НАРОДНОЕ, 0 км",0,1,2


## 4.2 Разработка API

Метод котрый возвращает все адреса по введенному региону и уровню опасности

In [6]:
def city(name, metka):
    return list(df.loc[(df['properties.parent_region']  == name) & (df['KMeans'] == metka), 'properties.address'])

Метод для получения единого фрейма данных из нескольких файлов

In [7]:
def filePath(path):
    allfile = pd.DataFrame()    
    for filename in glob.glob(path + '/*.csv'):
        data = pd.read_csv(filename)
        allfile.append(data)
    allfile = allfile[['properties.dead_count', 'properties.injured_count', 'properties.participants_count']]
    return allfile

Метод для предсказания меток

In [8]:
def prediction(X):
    scaler = StandardScaler()
    sc = scaler.fit_transform(df.select_dtypes(exclude=['object', 'datetime64', 'string']))
    pred = model.predict(X)
    return pred

## 4.1 Разработка бота

In [9]:
while True:
    inp = input('User: ').lower()
    if inp == 'город':
        print('Bot:  пример = Москва')
        name = input('\nBot: Введите город\nUser: ')
        print('\nBot:  Уровни опасности:\nТребует немедленного вмешательства-0\nТребует вмешательства-1\nВмешательство не требуется-2')
        metka = input('\nBot:  Введите уровень опасности\nUser: ')
        if metka == '0':
            ct = city(name, 'Требует немедленного вмешательства')
            cnt = int(input('\nBot:  Введите сколько адресов вы хотите увидеть\nUser: '))
            print('\n')
            for i in range(0, cnt):
                print('Bot:  '+ct[i])
        elif metka == '1':
            ct = city(name, 'Требует вмешательства')
            cnt = int(input('\nBot:  Введите сколько адресов вы хотите увидеть\nUser: '))
            print('\n')
            for i in range(0, cnt):
                print('Bot:  '+ct[i])
        elif metka == '2':
            ct = city(name, 'Вмешательство не требуется')
            cnt = int(input('\nBot:  Введите сколько адресов вы хотите увидеть\nUser: '))
            print('\n')
            for i in range(0, cnt):
                print('Bot:  '+ct[i])
    
    
    elif inp == 'предсказание':
        X = []
        daed = [int(input('\nBot:  Введите количество поигбших\nUser: '))]
        injured = [int(input('\nBot:  Введите количество пострадавших\nUser: '))]
        patr = [int(input('\nBot:  Введите количество участниклов дтп\nUser: '))]
        d = {'properties.dead_count':daed, 'properties.injured_count':injured, 'properties.participants_count':patr}
        X = pd.DataFrame(data = d)
        pr = prediction(X)
        print('Bot:  '+pr[0])
    
    
    if inp == 'дата':
        print(r'Bot:  пример = C:\files\data')
        path = input('User:  Введите путь к папке с файлами ')
        allfile = filePath(path)
        prall = prediction(allfile)
        allfile['Предсказание'] = prall
        
        
    elif inp == 'выход':
        break

User: город
Bot:  пример = Москва

Bot: Введите город
User: Москва

Bot:  Уровни опасности:
Требует немедленного вмешательства-0
Требует вмешательства-1
Вмешательство не требуется-2

Bot:  Введите уровень опасности
User: 0

Bot:  Введите сколько адресов вы хотите увидеть
User: 10


Bot:  г Москва, проезд Иртышский 1-й, 2, строение 8
Bot:  г Москва, ул Шереметьевская, 85
Bot:  г Москва, ул Болотниковская, 33 корп. 1
Bot:  г Москва, ул Совхозная, 12
Bot:  тер СНТ Полесье, 47
Bot:  г Москва, пр-кт Зеленый, 13
Bot:  г Москва, ш Дмитровское, 89
Bot:  г Москва, км МКАД 94-й, 3999м
Bot:  г Москва, проезд Коломенский, 17
Bot:  г Москва, ул Красного Маяка, 16
User: выход


In [10]:
#'C:/Users/Force/ws1/second step/4/data'